<a href="https://colab.research.google.com/github/pulavartivinay/Face_Mask_Detection/blob/model/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# start

In [4]:
# installs

In [5]:
# imports
import tensorflow as tf
import cv2
from keras import backend as K
from google.colab.patches import cv2_imshow

In [3]:
# loading dataset
# https://www.kaggle.com/ashishjangra27/face-mask-12k-images-dataset

!gdown --id 1NOzYPR3zAS8e2EFdRvjYpSBxg9RaDfxe #loading the dataset from google drive link to colab
!unzip /content/face-mask-12k.zip &> /dev/null #unzipping
!rm -rf /content/face-mask-12k.zip &> /dev/null #deleting the zip file
!rm -rf __MACOSX &> /dev/null

Downloading...
From: https://drive.google.com/uc?id=1NOzYPR3zAS8e2EFdRvjYpSBxg9RaDfxe
To: /content/face-mask-12k.zip
351MB [00:01, 224MB/s]


In [7]:
# Preprocessing the data set into list of images -- Vinay
# https://keras.io/api/preprocessing/image/#image_dataset_from_directory-function

train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/face-mask-12k/Train",
    labels="inferred",
    label_mode="binary",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(256, 256),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
)

val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/face-mask-12k/Validation",
    labels="inferred",
    label_mode="binary",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(256, 256),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
)

print(len(train_dataset))
print(len(val_dataset))


Found 10000 files belonging to 2 classes.
Found 800 files belonging to 2 classes.
313
25


In [ ]:
# Model(Transfer Learning) (optional)

In [8]:
# Main Model 

def get_model(input_shape):
  inputs = tf.keras.Input(shape=input_shape)  #input layer
  layer1 = tf.keras.layers.Conv2D(
      filters=2,
      kernel_size=4,
      strides=(1, 1),
      padding="valid",
      data_format=None,
      dilation_rate=(1, 1),
      groups=1,
      activation='relu',
      use_bias=True,
      kernel_initializer="glorot_uniform",
      bias_initializer="zeros",
      kernel_regularizer=None,
      bias_regularizer=None,
      activity_regularizer=None,
      kernel_constraint=None,
      bias_constraint=None,
  )(inputs)
  layer2 = tf.keras.layers.Flatten()(layer1)
  outputs = tf.keras.layers.Dense(
      1,
      activation='sigmoid',
      use_bias=True,
      kernel_initializer="glorot_uniform",
      bias_initializer="zeros",
      kernel_regularizer=None,
      bias_regularizer=None,
      activity_regularizer=None,
      kernel_constraint=None,
      bias_constraint=None,
  )(layer2)

  model = tf.keras.models.Model(inputs=inputs, outputs=outputs)
  model.summary()
  # plot_model(model, to_file='model_basic_with_updated_conv_and_dense_layers.png')
  return model

In [9]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [10]:
# Model Compilation -- Vinay
# https://keras.io/api/models/model_training_apis/

Main_Model = get_model((256,256,3))
Main_Model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=['acc',f1_m,precision_m, recall_m],
)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 253, 253, 2)       98        
_________________________________________________________________
flatten (Flatten)            (None, 128018)            0         
_________________________________________________________________
dense (Dense)                (None, 1)                 128019    
Total params: 128,117
Trainable params: 128,117
Non-trainable params: 0
_________________________________________________________________


In [11]:
# Training -- Vipul
# https://keras.io/api/models/model_training_apis/

Main_Model.fit(
    train_dataset,
    epochs=1,
    verbose=1,
)

313/313 [==============================] - 205s 650ms/step - loss: 1.3249 - acc: 0.7190 - f1_m: 0.7277 - precision_m: 0.7034 - recall_m: 0.7780


In [ ]:
# Adding call Back Functions(optional)
# https://medium.com/@thongonary/how-to-compute-f1-score-for-each-epoch-in-keras-a1acd17715a2

In [12]:
# model evaluation
# https://keras.io/api/models/model_training_apis/

Main_Model.evaluate(
    val_dataset,
    batch_size=32,
    verbose=1,
)

25/25 [==============================] - 7s 244ms/step - loss: 0.5966 - acc: 0.7300 - f1_m: 0.7643 - precision_m: 0.6770 - recall_m: 0.8926


[0.596629798412323,
 0.7300000190734863,
 0.764309823513031,
 0.677000105381012,
 0.8925660848617554]

In [ ]:
# Predictions of single random image -- Vipul
# https://keras.io/api/models/model_training_apis/

In [ ]:
# Predictions of Validation data set(Dev) -- Vinay

for images, label in val_dataset:
    prediction = Main_Model.predict(images, verbose=1)
    for i in range(len(prediction)):
      image = cv2.cvtColor(images[i].numpy(), cv2.COLOR_BGR2RGB)
      cv2_imshow(image)
      if(prediction[i] >= 0.5):
        p = 1
      else:
        p = 0
      print(p,label[i])

In [19]:
# Drawing the train and dev Loss and accuracy of the model, visualising too -- Vipul

In [ ]:
# Calculating the F1-Score of the Model on Dev data set -- Vinay
# https://datascience.stackexchange.com/questions/45165/how-to-get-accuracy-f1-precision-and-recall-for-a-keras-model
# https://medium.com/@thongonary/how-to-compute-f1-score-for-each-epoch-in-keras-a1acd17715a2

In [ ]:
# Visualisation of the F1-Score on Dev data set-- Vinay